In [1]:
import os

from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

import sys
import asyncio

if sys.platform == "win32":
    asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())

In [2]:
SERVER_URL = "0.0.0.0:8000"
TRUE_PATH = "src\\data\\true"
GENERATED_PATH = "src\\data\\generated"
METHODS = ["prompt_injection", "iframe", "obfuscation", "true", "htmlAppend", "shadowRootOpen", "shadowRootClose"]  # "random_elements"

In [3]:
folders = [
    f for f in os.listdir(TRUE_PATH) if os.path.isdir(os.path.join(TRUE_PATH, f))
]

# check if generated folder exists if not create folder
if not os.path.exists(GENERATED_PATH):
    os.makedirs(GENERATED_PATH)

In [ ]:
import os
from bs4 import BeautifulSoup
from playwright.sync_api import sync_playwright
import multiprocessing


def save_content(content: str, save_folder_path: str) -> None:
    soup = BeautifulSoup(content, "html.parser")
    print('here')
    with open(save_folder_path, "w") as f:
        print(save_folder_path)
        f.write(str(soup))


def scrape_page(page: object, file_path: str, method: str, save_folder_path: str) -> None:
    if method == "true":
        url = f"http://{SERVER_URL}?file_path={file_path}"
    else:
        url = f"http://{SERVER_URL}?file_path={file_path}&method={method}"

    try:
        page.goto(url, timeout=60000, wait_until="networkidle")
        content = page.content()
        save_content(content, save_folder_path)
    except Exception as e:
        print(f"Error scraping {url}: {e}")


def run_scraper() -> None:
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        context = browser.new_context()
        page = context.new_page()

        for folder in folders:
            files = [
                f
                for f in os.listdir(os.path.join(TRUE_PATH, folder))
                if os.path.isfile(os.path.join(TRUE_PATH, folder, f))
            ]
            for file in files:
                file_path = os.path.join(folder, file)
                for method in METHODS:
                    save_folder_path = os.path.join(GENERATED_PATH, folder, method)
                    if not os.path.exists(save_folder_path):
                        os.makedirs(save_folder_path)
                    save_folder_path = os.path.join(save_folder_path, file)
                    print(save_folder_path)
                    file_path = os.path.join(TRUE_PATH, file_path)
                    scrape_page(page, file_path, method, save_folder_path)

        browser.close()

    print("Scraping completed.")


def main():
    run_scraper()


if __name__ == "__main__":
    main()


Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.